In [2]:
%load_ext autoreload
%autoreload 2
import os
import zipfile

from metal.mmtl.bert_tasks import create_tasks
from metal.mmtl.metal_model import MetalModel

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### Config parameters

In [32]:
# where to save tsv files
submission_dir = 'submission_02_20_19_0'
# set to 'dev' to compute validation metrics on dev set or 'test' to write submissions in the submission_dir
eval_split = 'test'
max_datapoints = -1

### Define which models weights to use

In [33]:
# model config (needs to be the same as parameters used for training)
bert_model = "bert-base-uncased"
max_len = 256
bert_output_dim = 768
dl_kwargs = {"batch_size": 32, "shuffle": False}

In [34]:
# 02/18/19
model_paths = {
    'QNLI': '/dfs/scratch0/chami/metal/metal/mmtl/checkpoints/15-2-2019/QNLI_21_28_59/',
    'STSB': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/14-2-2019/STSB_17_59_04/',
    'SST2': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/14-2-2019/SST2_00_01_36/',
    'COLA': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/14-2-2019/COLA_19_48_08/',
    'RTE':'/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/RTE_22_52_53/',
    'WNLI': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/WNLI_23_05_55/',
    'QQP': '/dfs/scratch1/senwu/mmtl/logs/checkpoints/16-2-2019/QQP_SAN_01_48_40',
    'MRPC':'/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/MRPC_23_21_58/',
    'MNLI': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/15-2-2019/MNLI_22_13_38/'
}

# 02/19/19
model_paths = {
    'MNLI_SAN': '/dfs/scratch1/senwu/mmtl/logs/checkpoints/17-2-2019/MNLI_SAN_02_27_41/',
    'QNLI': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/QNLI_09_56_46/',
    'STSB': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/STSB_09_46_46/',
    'SST2': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/SST2_06_01_35/',
    'COLA': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/COLA_05_49_39/',
    'RTE':'/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/RTE_06_32_37/',
    'WNLI': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/WNLI_06_38_32/',
    'QQP': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/QQP_06_47_48/',
    'MRPC':'/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/MRPC_09_40_25/'
}

# 02/19/19
model_paths = {
    'MNLI_SAN': '/dfs/scratch1/senwu/mmtl/logs/checkpoints/17-2-2019/MNLI_SAN_02_27_41/',
    'QNLI': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/QNLI_09_56_46/',
    'STSB': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/STSB_09_46_46/',
    'SST2': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/SST2_06_01_35/',
    'COLA': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/COLA_05_49_39/',
    'RTE':'/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/RTE_06_32_37/',
    'WNLI': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/WNLI_06_38_32/',
    'QQP': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/QQP_06_47_48/',
    'MRPC':'/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/MRPC_09_40_25/'
}

# 02/19/19
model_paths = {
    'MNLI_SAN': '/dfs/scratch1/senwu/mmtl/logs/checkpoints/17-2-2019/MNLI_SAN_02_27_41/',
    'QNLI': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/QNLI_09_56_46/',
    'STSB': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/STSB_09_46_46/',
    'SST2': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/SST2_06_01_35/',
    'COLA': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/COLA_05_49_39/',
    'RTE':'/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/RTE_06_32_37/',
    'WNLI': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/WNLI_06_38_32/',
    'QQP': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/QQP_06_47_48/',
    'MRPC':'/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/MRPC_09_40_25/'
}

# 02/19/19 (sen's models)
model_paths = {
    #'WNLI': '/dfs/scratch1/senwu/mmtl/logs/ckpt/14-2-2019/WNLI_SAN_09_50_37/',
    #'MNLI_SAN': '/dfs/scratch1/senwu/mmtl/logs/checkpoints/17-2-2019/MNLI_SAN_02_27_41/',
    'RTE_SAN': '/dfs/scratch1/senwu/mmtl/logs/2019_02_19/RTE_SAN_11_44_21/',
    #'QQP_SAN': '/dfs/scratch1/senwu/mmtl/logs/checkpoints/16-2-2019/QQP_SAN_01_48_40/',
    #'MRPC_SAN':'/dfs/scratch1/senwu/mmtl/logs/checkpoints/16-2-2019/MRPC_SAN_10_36_39/',
    #"QNLI": '/dfs/scratch0/chami/metal/metal/mmtl/checkpoints/19-2-2019/QNLI_02_09_46/'
}

# best models based on val scores 
model_paths = {
    #'QNLI': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/QNLI_09_56_46/',
    #'STSB': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/14-2-2019/STSB_17_59_04/',
    #'STSB': '/dfs/scratch0/chami/metal/logs/2019_02_21/STSB_00_20_13',
    #’SST2': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/14-2-2019/SST2_00_01_36/',
    #'SST2': '/dfs/scratch0/chami/metal/metal/mmtl/checkpoints/19-2-2019/SST2_02_01_09',
    #'COLA': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/COLA_05_49_39/',
    #'COLA': '/dfs/scratch0/mccreery/mmtl/logs/2019_02_19/4_tuned_COLA/logdir/2019_02_19/COLA_20_56_07',
    'COLA': '/dfs/scratch0/chami/metal/logs/2019_02_20/COLA_23_41_22/',
    #'RTE':'/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/RTE_22_52_53/',
    "RTE": "/dfs/scratch0/chami/metal/logs/2019_02_20/RTE_21_25_34/",
    #'WNLI': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/WNLI_23_05_55/',
    #'QQP': '/dfs/scratch1/senwu/mmtl/logs/checkpoints/16-2-2019/QQP_SAN_01_48_40',
    #'MRPC':'/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/13-2-2019/MRPC_23_21_58/',
    'MRPC_SAN': '/dfs/scratch1/senwu/mmtl/logs/2019_02_19/MRPC_SAN_11_57_55/',
    #'MRPC': '/dfs/scratch0/chami/metal/logs/2019_02_20/MRPC_23_56_29'
    #’MNLI': '/dfs/scratch0/vschen/metal-mmtl/logs/test_logs/15-2-2019/MNLI_22_13_38/'
}

### Helper functions

In [35]:
def save_tsv(predictions, task_name, state, submission_dir):
    task_to_name_dict = {
        "QNLI": "QNLI",
        "STSB": "STS-B",
        "SST2": "SST-2",
        "COLA": "CoLA",
        "MNLI" : {"mismatched": "MNLI-mm", "matched": "MNLI-m"},
        "MNLI_SAN" : {"mismatched": "MNLI-mm", "matched": "MNLI-m"},
        "RTE": "RTE",
        "RTE_SAN": "RTE",
        "WNLI": "WNLI",
        "WNLI_SAN": "WNLI",
        "QQP": "QQP",
        "QQP_SAN": "QQP",
        "MRPC": "MRPC",
        "MRPC_SAN": "MRPC"        
    }
    if isinstance(task_to_name_dict[task_name], dict):
        file_name = task_to_name_dict[task_name][state]
    else:
        file_name = task_to_name_dict[task_name]
    file_path = os.path.join(submission_dir, f"{file_name}.tsv")
    with open(file_path, 'w') as f:
        f.write('index\tprediction\n')
        for idx, pred in enumerate(predictions):
            f.write(f'{int(idx)}\t{str(pred)}\n')
    print("Saved TSV to: ", file_path)

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.split('.')[-1] == 'tsv':
                ziph.write(os.path.join(root, file))

### Compute and save predictions

In [36]:
# create submission directory if not found
if not os.path.isdir(submission_dir):
    os.mkdir(submission_dir)

all_scores = []

for task_name, model_dir in model_paths.items():
    print(model_dir)
    if 'MNLI' in task_name:
        states = ['mismatched', 'matched']
        splits = [f'{eval_split}_{state}' for state in states]
    else: 
        states = [None]
        splits = [eval_split]

    # create model
    task = create_tasks(
        task_names=[task_name],
        bert_model=bert_model,
        max_len=max_len,
        dl_kwargs=dl_kwargs,
        bert_output_dim=bert_output_dim,
        splits=splits,
        max_datapoints=max_datapoints,
    )[0]
    
    # load model weights 
    model_path = os.path.join(model_dir, 'best_model.pth')
    model = MetalModel([task], verbose=False, device=0)
    model.load_weights(model_path)
    model.eval()
    
    if eval_split == 'dev':
        # just compute evaluation metrics for debugging
        score = task.scorer.score(model, task)
        all_scores.append(score)
            
        print(score)

    else:
        
        for i, split in enumerate(splits):
            
            # predict on eval_set
            Y, Y_probs, Y_preds = model._predict_probs(task, split=split, return_preds=True)
        
            inv_label_fn = task.data_loaders[split].dataset.inv_label_fn
            predicted_labels = [inv_label_fn(pred) for pred in Y_preds]
        
            if task_name != 'STSB':
                predicted_labels = [inv_label_fn(pred) for pred in Y_preds]
            else:
                # STSB is a regression task so we operate directly on predictions
                predicted_labels = [round(inv_label_fn(pred), 3) for pred in Y_probs.flatten()]
    
            # save predictions on test set for submission
            save_tsv(predicted_labels, task_name, states[i], submission_dir)

for score in all_scores:
    print(score)

/dfs/scratch0/chami/metal/logs/2019_02_20/COLA_23_41_22/
Using random seed: 5490.
Loading COLA Dataset



Saved TSV to:  submission_02_20_19_0/CoLA.tsv
/dfs/scratch0/chami/metal/logs/2019_02_20/RTE_21_25_34/
Using random seed: 323640.
Loading RTE Dataset



Saved TSV to:  submission_02_20_19_0/RTE.tsv
/dfs/scratch1/senwu/mmtl/logs/2019_02_19/MRPC_SAN_11_57_55/
Using random seed: 355122.
Loading MRPC Dataset



Saved TSV to:  submission_02_20_19_0/MRPC.tsv


**In case one is too lazy to zip manually**

In [ ]:
zipf = zipfile.ZipFile(f'{submission_dir}.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir(submission_dir, zipf)
zipf.close()